In [1]:
import requests # library to handle requests
import bs4
import pandas as pd
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

CLIENT_ID = 'OOL3CJVWOLDGNU01D1VMCWKV2M1MQN5EJVXO13BBHWZPKGYR' # your Foursquare ID
CLIENT_SECRET = 'VZWPT2ARYG2CTCL1QSUTIUM2B5MFIIH4HOEJUGSUN15CXM3U' # your Foursquare Secret
ACCESS_TOKEN = 'KEZAPAWX2SID04SJG51EVSH5332ZQX1OXTFXMNEIYPWB2TBU' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100 # A default Foursquare API limit value

In [2]:
#Scrape the wikipeadia page using requests and BeautifulSoup
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url)
soup = bs4.BeautifulSoup(results.text, 'lxml')

#Create an array for extract the relevant using information
PostalCode = []
Borough = []
Neighborhood = []

#Extract elements from the table 
table = soup.find("table",class_ =  "wikitable sortable")


#convert elemnts into string
table = table.get_text()

#convert the elemnts of the table from string to list and split the list based on \n
table = table.split('\n')

#Remove white space from the list
str_list = list(filter(None, table))


In [3]:
# Extract PostalCode from the list
PostalCode = str_list[0:len(str_list):3]

# Extract Borough from the list
Borough = str_list[1:len(str_list):3]

# Extract Neighborhood from the list
Neighborhood = str_list[2:len(str_list):3]

# Convert list into Series
se1 = pd.Series(PostalCode)
se2 = pd.Series(Borough)
se3 = pd.Series(Neighborhood)

In [4]:
# Create a dataframe and add Series to the dataframe
Dataframe = pd.concat([se1,se2,se3], axis=1)

# Rename 
Dataframe.columns = ["PostalCode", "Borough", "Neighborhood"]

In [5]:
# Remove "Not Assigned" within the column "Borough"
df = Dataframe[Dataframe["Borough"] != "Not assigned"].reset_index(drop=True)

# Remove the first row as it contains the column name of the table
df = df.drop(df.index[0])

In [9]:
# Extract latitiude and longtitude data from the given CSV file
locgeo_df = pd.read_csv('https://cocl.us/Geospatial_data', index_col='Postal Code')


toronto_data = df.join(locgeo_df,  on='PostalCode')

In [10]:
toronto_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
1,M3A,North York,Parkwoods,43.753259,-79.329656
2,M4A,North York,Victoria Village,43.725882,-79.315572
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
5,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
99,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
100,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
101,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
102,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
